
# GENBAIT Tutorial

This notebook provides a detailed walkthrough of the GENBAIT Python package for selecting optimal baits in BioID experiments using Genetic Algorithms (GA) and various clustering metrics. We will guide you through loading BioID data, running a genetic algorithm for bait selection, and evaluating the results using different NMF-derived and non NMF-derived metrics. 



## Importing required modules

In [1]:
import genbait as gb
import warnings
warnings.filterwarnings('ignore')


## Setting Input, Output Directories, and Parameters for NMF and GA

In this section, we define the input and output directories for file handling, as well as key parameters for Non-negative Matrix Factorization (NMF) and the Genetic Algorithm (GA) used in feature (bait) selection.

- **input_directory**: This variable defines the path where the input files, such as the SAINT file and other data, are stored.
- **output_directory**: This variable specifies the location where the output files will be saved after processing, including the results of the genetic algorithm and NMF computations.

- **primary_baits**: A list of primary baits that can be used to filter the dataset. If set to `None`, all baits from the dataset will be used. You can uncomment this line to provide specific bait proteins.

- **n_components**: Specifies the number of NMF components (latent features). This parameter controls how many underlying factors will be extracted from the BioID dataset. Typically, these components represent key biological factors.

- **n_baits_to_select**: Defines the number of baits (features) to be selected by the genetic algorithm. The goal of the GA is to find an optimal subset of baits that retain the biological significance of the dataset.

- **subset_range**: This is the range of the number of baits that can be selected by the genetic algorithm. It is calculated based on the `n_baits_to_select` value to ensure that the number of selected baits remains within a valid range (neither too many nor too few).
  - Function: `calculate_subset_range`
  - Input: `n_baits_to_select` (the desired number of baits to select).
  - Output: A tuple representing the valid range of baits that the GA can select.


In [15]:
input_directory = 'data/input_files/'  
output_directory = 'data/output_files/'  

# primary_baits = ['Gene1', 'Gene2', 'Gene3'] 

n_components = 20  
n_baits_to_select = 50  

subset_range = gb.calculate_subset_range(n_baits_to_select)  

## Loading and Normalizing the BioID Data

In this step, we load and preprocess the BioID data using the `load_bioid_data` function. This function reads the SAINT file and performs preprocessing steps such as calculating average control intensities, correcting for control values, and normalizing the data.

- **Function**: `load_bioid_data`
- **Parameters**:
  - `input_file_path`: The path to the SAINT file (located in the `input_directory`).
  - `output_file_directory`: The directory where the processed and normalized data will be saved. The normalized DataFrame will also be stored as a CSV file for further use.
  
- **Process**:
  - **Step 1**: The SAINT file is read and baits (proteins) are processed.
  - **Step 2**: Controls are averaged, and prey spectral counts are adjusted by subtracting these control averages.
  - **Step 3**: The data is filtered by a False Discovery Rate (FDR) threshold, pivoted to a matrix format (baits as rows, preys as columns), and normalized using MinMax scaling.

- **Output**: 
  - Returns the normalized DataFrame (`df_norm`), which can be used in subsequent analyses such as feature selection and clustering.



In [9]:
df_norm = gb.load_bioid_data(
    input_file_path=f'{input_directory}saint-latest.txt',
    output_file_directory=output_directory
)

In [10]:
df_norm

PreyGene,AAAS,AAK1,AAR2,AARS2,AASDH,AASS,AATF,ABCA3,ABCB1,ABCB10,...,ZNF830,ZNF850,ZNF91,ZRANB2,ZSCAN18,ZSCAN21,ZW10,ZWINT,ZYX,ZZZ3
Bait,,,,,,,,,,,,,,,,,,,,,
AARS2,0.0,0.000000,0.0,0.0,0.0,0.347015,0.000000,0.0,0.0,0.266667,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
ACBD5,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
ACTB,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.384215,0.0
ACTR1A,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
ACTR2,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VCL,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.278706,0.0
VIM,0.0,0.123211,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
ZFPL1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0


## Running the Genetic Algorithm

In this step, we use the `run_ga` function to run a genetic algorithm (GA) for selecting an optimal subset of features (baits) from the BioID dataset. The GA evolves a population of possible solutions over multiple generations to find the best subset of baits.

- **Function**: `run_ga`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `n_components`: The number of NMF components used in the analysis.
  - `subset_range`: The range of the number of selected baits (features).
  - `population_size`: The number of individuals in the GA population. In this case, it's set to 50.
  - `n_generations`: The number of generations for which the GA will run. In this case, it's set to 3.

- **Returns**:
  - `pop`: The final population of individuals after the GA has evolved over the specified generations.
  - `logbook`: A logbook that contains statistical information about the evolution process, including metrics like the average fitness, minimum fitness, and maximum fitness for each generation.
  - `hof`: The Hall of Fame, which contains the best individuals (i.e., subsets of baits) discovered during the GA run.




In [12]:
pop, logbook, hof = gb.run_ga(
    df_norm=df_norm,
    n_components=n_components,
    subset_range=subset_range,
    population_size=50,
    n_generations=3
)

gen	nevals	avg      	std     	min    	max     
0  	50    	-0.496768	0.460882	-1.5312	0.646729
1  	33    	-0.156691	0.408448	-1.32808	0.668128
2  	33    	0.0120869	0.35166 	-1.01133	0.662135
3  	28    	0.199545 	0.326013	-0.481102	0.709199


## Saving GA results
Here, we save the results of the GA using the `save_ga_results` function, which saves the population, logbook, and Hall of Fame to specified file paths.

- **Parameters**:
  - `population`: The final population after the GA run.
  - `logbook`: The logbook tracking the evolution of the population.
  - `hof`: Hall of Fame containing the best individuals found by the GA.
  - `pop_file_path`, `logbook_file_path`, `hof_file_path`: Paths to save the files.





In [13]:
gb.save_ga_results(
    population=pop,
    logbook=logbook,
    hof=hof,
    pop_file_path=f"{output_directory}/popfile.pkl",
    logbook_file_path=f"{output_directory}/logbookfile.pkl",
    hof_file_path=f"{output_directory}/hoffile.pkl"
)

## Loading GA results
Here we can load previously saved population and logbook from their respective file paths

In [12]:
# population, logbook, hof = gb.load_ga_results(
#     pop_file_path=f"{output_directory}/popfile.pkl",
#     logbook_file_path=f"{output_directory}/logbookfile.pkl",
#     hof_file_path=f"{output_directory}/hoffile.pkl"
# )

## Calculating GA result

In this step, we use the `calculate_ga_results` function to retrieve the best subset of baits selected by the genetic algorithm and calculate the NMF correlation between the original and selected subset. This function returns the selected baits and their corresponding NMF correlation values.

- **Function**: `calculate_ga_results`
- **Parameters**:
  - `hof`: The Hall of Fame from the genetic algorithm, which contains the best individuals (subsets of baits).
  - `df_norm`: The normalized BioID data (DataFrame).
  - `n_components`: The number of NMF components used in the analysis.
  
- **Returns**:
  - `selected_baits`: The list of baits selected by the genetic algorithm as the optimal subset.
  - `mean_nmf_correlation`: The mean correlation between the NMF basis matrix of the original dataset and the subset dataset.
  - `all_nmf_correaltions`: A list of correlation values for each NMF component.




In [13]:
selected_baits, mean_nmf_correlation, all_nmf_correaltions = gb.calculate_ga_results(hof, df_norm, n_components)

In [14]:
selected_baits

['AKAP1',
 'ANK3',
 'ASF1A',
 'ATP2A1',
 'CALR3',
 'CBX3',
 'CLTA',
 'COX8A_target',
 'CS',
 'CTNNA1',
 'CYP2C1_sigseq',
 'DHX40',
 'EBAG9',
 'EPB41L4A_Cterm',
 'FLOT1',
 'GFAP',
 'GORASP1',
 'HNRNPA1',
 'LNPK_Nterm',
 'LRRC59_Nterm',
 'MAPRE1',
 'MARCKS_target',
 'MLLT4',
 'NDC80',
 'NOP56',
 'NPM1',
 'NUP155_Nterm',
 'PARP1',
 'PRPF8',
 'RAB5A',
 'RAB5C',
 'RAB9A',
 'RHOB',
 'RPL31',
 'RPS24',
 'SEC61B_Cterm',
 'SFXN1',
 'STX7',
 'SYNE3_Cterm',
 'SYNE3_Nterm',
 'TERF2IP',
 'TRIM36_Nterm',
 'TUBA1A',
 'TUBA4A',
 'TUBB']

In [15]:
mean_nmf_correlation

0.7091985502317943

In [16]:
all_nmf_correaltions

array([0.79226554, 0.85351557, 0.15611431, 0.93416164, 0.96406943,
       0.64759528, 0.96251264, 0.95233955, 0.77993336, 0.93445555,
       0.51533336, 0.03571605, 0.99051509, 0.48101593, 0.40425275,
       0.86361245, 0.93234166, 0.81742301, 0.17157149, 0.99522634])

## Calculating NMF Cosine similarity

In this step, we use the `calculate_nmf_cosine_similarity` function to compute the cosine similarity between the original and selected subset baits. This similarity gives insight into how well the selected baits represent the original data.

- **Function**: `calculate_nmf_cosine_similarity`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The subset of baits selected by the genetic algorithm.
  - `n_components`: The number of NMF components used in the analysis.
  
- **Returns**:
  - `mean_nmf_cos_similarity_score`: The mean cosine similarity score between the NMF components of the original dataset and the selected subset.
  - `all_nmf_cos_similarity_scores`: A list of cosine similarity scores for each NMF component, showing the similarity for individual components.



In [17]:
mean_nmf_cos_similarity_score, all_nmf_cos_similarity_scores = gb.calculate_nmf_cosine_similarity(df_norm, selected_baits, n_components)

In [18]:
mean_nmf_cos_similarity_score

0.7245421537225744

In [19]:
all_nmf_cos_similarity_scores

array([0.8125022 , 0.86230352, 0.19511033, 0.93815166, 0.96538689,
       0.67531421, 0.96437863, 0.95253445, 0.78516601, 0.93861229,
       0.54977818, 0.08476043, 0.99135444, 0.52194027, 0.42729313,
       0.86739639, 0.93356794, 0.82899058, 0.20088615, 0.99541539])

## Calculating NMF KL divergence


In this step, we use the `calculate_nmf_kl_divergence` function to compute the Kullback-Leibler (KL) divergence between the NMF components of the original data and the selected baits. KL divergence measures how one probability distribution diverges from a second, expected probability distribution. This is useful for understanding the differences between the original data and the subset selected by the genetic algorithm.

- **Function**: `calculate_nmf_kl_divergence`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The list of baits (features) selected by the GA.
  - `n_components`: The number of NMF components used in the analysis.

- **Returns**:
  - `mean_nmf_kl_divergence_score`: The average KL divergence score across all NMF components.
  - `all_nmf_kl_divergence_scores`: The individual KL divergence scores for each component.


In [20]:
mean_nmf_kl_divergence_score, all_nmf_kl_divergence_scores = gb.calculate_nmf_kl_divergence(df_norm, selected_baits, n_components)

In [21]:
mean_nmf_kl_divergence_score

6.137286525421461

In [22]:
all_nmf_kl_divergence_scores

array([ 3.44579802,  3.98665558, 16.24365059,  1.65621653,  1.39011435,
        6.24679521,  2.3820665 ,  3.7649966 ,  7.06126624,  2.62323316,
        8.99116679, 18.10645507,  0.36588091,  5.95561483,  9.76928944,
        6.23863719,  3.1108764 ,  3.90540797, 16.65477775,  0.84683137])

## Calculating NMF GO Jaccard Index

In this step, we use the `calculate_nmf_go_jaccard` function to compute the Jaccard Index between the Gene Ontology (GO) Cellular Component (CC) terms associated with the original data and the selected baits. The Jaccard Index is used to measure the similarity between two sets, specifically the overlap between GO terms of the original and selected baits.

- **Function**: `calculate_nmf_go_jaccard`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The list of baits (features) selected by the genetic algorithm.
  - `n_components`: The number of NMF components used in the analysis.

- **Returns**:
  - `mean_nmf_go_jaccard_index`: The average Jaccard index score across all NMF components.
  - `all_nmf_go_jaccard_indices`: The individual Jaccard index scores for each component.



In [23]:
mean_nmf_go_jaccard_index, all_nmf_go_jaccard_indices = gb.calculate_nmf_go_jaccard(df_norm, selected_baits, n_components=n_components)


In [24]:
mean_nmf_go_jaccard_index

0.5429466839122716

In [25]:
all_nmf_go_jaccard_indices

{2: 0.31868131868131866,
 6: 0.6785714285714286,
 4: 0.9393939393939394,
 5: 0.5151515151515151,
 9: 0.6947368421052632,
 0: 0.43005181347150256,
 1: 0.5423728813559322,
 7: 0.3942307692307692,
 3: 0.8783783783783784,
 8: 0.6461538461538462,
 11: 0.21495327102803738,
 13: 0.3783783783783784,
 16: 0.9361702127659575,
 12: 0.6363636363636364,
 18: 0.1794871794871795,
 15: 0.5393258426966292,
 17: 0.5080645161290323,
 10: 0.6190476190476191,
 14: 0.48333333333333334,
 19: 0.32608695652173914}

## Calculating NMF ARI (Adjusted Rand Index)

In this step, we use the `calculate_nmf_ari` function to compute the Adjusted Rand Index (ARI) between the clustering of the original data and the selected baits. ARI measures the similarity of the clustering assignments between two different sets, where a higher ARI indicates better alignment between the clusters.

- **Function**: `calculate_nmf_ari`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The list of baits (features) selected by the genetic algorithm.
  - `n_components`: The number of NMF components used in the analysis.

- **Returns**:
  - `nmf_ari_score`: The ARI score indicating how well the selected baits' clustering matches the clustering of the original data.



In [26]:
nmf_ari_score = gb.calculate_nmf_ari(df_norm, selected_baits, n_components)

In [27]:
nmf_ari_score

0.5116111477977049

## Calculating Remaining Preys Percentage and Count

In this step, we use the `calculate_remaining_preys_percentage` function to compute the percentage and count of preys (proteins) that remain in the selected subset of data after applying the genetic algorithm. This helps assess the coverage of preys in the selected baits.

- **Function**: `calculate_remaining_preys_percentage`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The list of baits (features) selected by the genetic algorithm.
  - `n_components`: The number of NMF components used in the analysis.

- **Returns**:
  - `remaining_preys_percentage`: The percentage of preys that remain in the selected baits.
  - `remaining_preys_count`: The total number of preys remaining in the selected subset.



In [28]:
remaining_preys_percentage, remaining_preys_count = gb.calculate_remaining_preys_percentage(df_norm, selected_baits, n_components)

In [29]:
remaining_preys_percentage

0.7174502712477396

In [30]:
remaining_preys_count

3174

## Calculating GO Retrieval Percentage

In this step, we use the `calculate_go_retrieval_percentage` function to calculate the Gene Ontology (GO) retrieval percentage for the selected baits. GO retrieval assesses the functional enrichment of the selected baits by evaluating how many of the preys are associated with known GO terms (specifically Cellular Component, or GO:CC) based on the provided GAF (Gene Annotation Format) file.

- **Function**: `calculate_go_retrieval_percentage`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The list of baits (features) selected by the genetic algorithm.
  - `gaf_path`: The file path to the GAF file, which contains GO annotations for various genes/proteins.

- **Returns**:
  - `go_retrieval_percentage`: The percentage of preys in the selected baits that are associated with known GO:CC terms.



In [33]:
go_retrieval_percentage = gb.calculate_go_retrieval_percentage(df_norm, selected_baits, gaf_path=f'{input_directory}goa_human.gaf')

In [34]:
go_retrieval_percentage

90.70660522273425

## Calculating Leiden ARI (Adjusted Rand Index)

In this step, we use the `calculate_leiden_ari` function to calculate the Adjusted Rand Index (ARI) for clustering the selected baits using the Leiden algorithm. The Leiden algorithm is a community detection algorithm commonly used for graph-based clustering. By calculating the ARI, we can compare how well the clustering from the selected baits corresponds to the clustering of the entire dataset across different resolutions. ARI measures the similarity between two clusterings, comparing the clustering of the full dataset with the clustering of the selected subset of baits.

- **Function**: `calculate_leiden_ari`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The list of baits (features) selected by the genetic algorithm.
  - `resolutions`: List of resolutions for the Leiden clustering algorithm. Higher resolutions lead to more clusters.
  - `seed`: (Optional) Random seed for reproducibility (default is 4).

- **Returns**:
  - `leiden_results`: A dictionary where the keys are resolution values, and the values are the ARI scores between the clustering of the original data and the selected baits.



In [35]:
leiden_results = gb.calculate_leiden_ari(df_norm, selected_baits, resolutions=[0.5, 1.0, 1.5])

In [36]:
for resolution, ari in leiden_results.items():
    print(f"Resolution: {resolution}, ARI: {ari}")

Resolution: 0.5, ARI: 0.5218981053123726
Resolution: 1.0, ARI: 0.5272409033404254
Resolution: 1.5, ARI: 0.4647442350933481


## Calculating GMM ARI (Adjusted Rand Index)

In this step, we use the `calculate_gmm_ari` function to calculate the Adjusted Rand Index (ARI) for Gaussian Mixture Model (GMM) clustering of the selected baits. The Gaussian Mixture Model (GMM) is used to cluster the data into a specified number of clusters. The ARI score measures the similarity between the clustering of the full dataset and the selected baits subset.

- **Function**: `calculate_gmm_ari`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The list of baits (features) selected by the genetic algorithm.
  - `cluster_numbers`: List of cluster numbers to fit the Gaussian Mixture Model (GMM). Each number represents how many clusters are formed.
  - `seed`: (Optional) Random seed for reproducibility (default is 4).

- **Returns**:
  - `gmm_results`: A dictionary where the keys are cluster numbers, and the values are the ARI scores comparing the clustering of the full dataset and the selected baits subset for each cluster number.


In [37]:
gmm_results = gb.calculate_gmm_ari(df_norm, selected_baits, cluster_numbers=[15, 20, 25, 30])

In [38]:
for cluster_number, ari in gmm_results.items():
    print(f"Cluster Number: {cluster_number}, ARI: {ari}")

Cluster Number: 15, ARI: 0.36254999357590995
Cluster Number: 20, ARI: 0.3834121704190322
Cluster Number: 25, ARI: 0.473268923990476
Cluster Number: 30, ARI: 0.3604236140421185


## Calculating GMM Mean Correlation

In this step, we use the `calculate_gmm_mean_correlation` function to compute the mean correlation values between the Gaussian Mixture Model (GMM) clustering of the full dataset and the selected baits subset. Gaussian Mixture Model (GMM) soft clustering is performed on both the full dataset and the selected baits subset for each specified cluster number. The function computes the mean correlation between the probability distributions of the original and subset clusters, indicating how well the selected baits maintain the clustering structure of the full dataset.

- **Function**: `calculate_gmm_mean_correlation`
- **Parameters**:
  - `df_norm`: The normalized BioID data (DataFrame).
  - `selected_baits`: The list of baits (features) selected by the genetic algorithm.
  - `cluster_numbers`: List of cluster numbers to fit the Gaussian Mixture Model (GMM). Each number represents how many clusters are formed.
  - `seed`: (Optional) Random seed for reproducibility (default is 4).

- **Returns**:
  - `gmm_corr_results`: A dictionary where the keys are cluster numbers and the values are the mean correlation values between the soft clustering of the full dataset and the selected baits subset for each cluster number.


  


In [39]:
gmm_corr_results = gb.calculate_gmm_mean_correlation(df_norm, selected_baits, cluster_numbers=[15, 20, 25, 30])

In [40]:
for cluster_number, mean_corr in gmm_corr_results.items():
    print(f"Cluster Number: {cluster_number}, Mean Correlation: {mean_corr}")

Cluster Number: 15, Mean Correlation: 0.2727451914291265
Cluster Number: 20, Mean Correlation: 0.3158209295933284
Cluster Number: 25, Mean Correlation: 0.31291794460313144
Cluster Number: 30, Mean Correlation: 0.27618404245265055
